In [4]:
from datasets import load_from_disk

is2_train = load_from_disk("/home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train_temp_2/smol-name_conversion-i2s_subtask-0_train")
is2_val = load_from_disk("/home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train_temp_2/smol-name_conversion-i2s_subtask-0_val")
is2_test = load_from_disk("/home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train_temp_2/smol-name_conversion-i2s_subtask-0_test")

In [5]:
len(is2_train), len(is2_val), len(is2_test)

(299890, 1496, 2993)

In [6]:
homo_lumo_gap = load_from_disk('/home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train/qm9_homo_lumo_gap_subtask-0_test')

In [7]:
homo_lumo_gap[0]

{'x': [[5, 0, 4, 5, 3, 0, 2, 0, 0],
  [5, 0, 3, 5, 0, 0, 1, 1, 1],
  [5, 0, 3, 5, 1, 0, 1, 1, 1],
  [7, 0, 2, 5, 0, 0, 1, 1, 1],
  [5, 0, 3, 5, 1, 0, 1, 1, 1],
  [5, 0, 3, 5, 0, 0, 1, 1, 1],
  [5, 0, 4, 5, 1, 0, 2, 0, 0],
  [5, 0, 4, 5, 3, 0, 2, 0, 0],
  [5, 0, 4, 5, 3, 0, 2, 0, 0]],
 'edge_index': [[0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 6, 8, 5, 1],
  [1, 0, 2, 1, 3, 2, 4, 3, 5, 4, 6, 5, 7, 6, 8, 6, 1, 5]],
 'edge_attr': [[0, 0, 0],
  [0, 0, 0],
  [3, 0, 1],
  [3, 0, 1],
  [3, 0, 1],
  [3, 0, 1],
  [3, 0, 1],
  [3, 0, 1],
  [3, 0, 1],
  [3, 0, 1],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [3, 0, 1],
  [3, 0, 1]],
 'label': '<FLOAT> <|-|><|0|><|.|><|2|><|1|><|3|><|2|> </FLOAT>',
 'input_mol_string': '<SELFIES> [C][C][=C][O][C][=C][Ring1][Branch1][C][Branch1][C][C][C] </SELFIES>',
 'task_subtask_pair': 'qm9_homo_lumo_gap',
 'instruction': 'Propose the HOMO-LUMO gap value given this molecule. <INPUT>'}

In [8]:
homo = load_from_disk('/home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train/qm9_homo_subtask-0_val')

In [9]:
homo[0]

{'x': [[7, 0, 1, 5, 0, 0, 1, 0, 0],
  [5, 0, 3, 5, 1, 0, 1, 0, 0],
  [5, 0, 4, 5, 0, 0, 2, 0, 1],
  [5, 0, 4, 5, 2, 0, 2, 0, 1],
  [6, 0, 3, 5, 0, 0, 2, 0, 1],
  [5, 0, 4, 5, 1, 0, 2, 0, 1],
  [5, 0, 4, 5, 2, 0, 2, 0, 1],
  [5, 0, 4, 5, 1, 0, 2, 0, 1],
  [5, 0, 4, 5, 2, 0, 2, 0, 1]],
 'edge_index': [[0,
   1,
   1,
   2,
   2,
   3,
   3,
   4,
   4,
   5,
   5,
   6,
   6,
   7,
   7,
   8,
   4,
   2,
   8,
   2,
   7,
   5],
  [1, 0, 2, 1, 3, 2, 4, 3, 5, 4, 6, 5, 7, 6, 8, 7, 2, 4, 2, 8, 5, 7]],
 'edge_attr': [[1, 0, 0],
  [1, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0]],
 'label': '<FLOAT> <|-|><|0|><|.|><|2|><|4|><|1|><|7|> </FLOAT>',
 'input_mol_string': '<SELFIES> [O][=C][C][C][N][Ring1][Ring1][C][C][C][Ring1][Ring1][C][Ring1][#Branch1] </SELFIES>',
 'task_subta

In [10]:
lumo = load_from_disk('/home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train/qm9_lumo_subtask-0_test')

In [11]:
lumo[0]

{'x': [[5, 0, 4, 5, 3, 0, 2, 0, 0],
  [5, 0, 4, 5, 2, 0, 2, 0, 0],
  [5, 0, 3, 5, 0, 0, 1, 0, 0],
  [7, 0, 1, 5, 0, 0, 1, 0, 0],
  [6, 0, 3, 5, 0, 0, 1, 0, 0],
  [5, 0, 4, 5, 3, 0, 2, 0, 0],
  [5, 0, 3, 5, 0, 0, 1, 0, 0],
  [7, 0, 1, 5, 0, 0, 1, 0, 0],
  [6, 0, 3, 5, 2, 0, 1, 0, 0]],
 'edge_index': [[0, 1, 1, 2, 2, 3, 2, 4, 4, 5, 4, 6, 6, 7, 6, 8],
  [1, 0, 2, 1, 3, 2, 4, 2, 5, 4, 6, 4, 7, 6, 8, 6]],
 'edge_attr': [[0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [1, 0, 1],
  [1, 0, 1],
  [0, 0, 1],
  [0, 0, 1],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 1],
  [0, 0, 1],
  [1, 0, 1],
  [1, 0, 1],
  [0, 0, 1],
  [0, 0, 1]],
 'label': '<FLOAT> <|-|><|0|><|.|><|0|><|0|><|6|><|4|> </FLOAT>',
 'input_mol_string': '<SELFIES> [C][C][C][=Branch1][C][=O][N][Branch1][C][C][C][=Branch1][C][=O][N] </SELFIES>',
 'task_subtask_pair': 'qm9_lumo',
 'instruction': '<INPUT> Based on the molecule given above, suggest the LUMO energy.'}

In [1]:
import pandas as pd
from datasets import load_from_disk
import re

# 1. Label 파싱 함수 정의 (특수 토큰을 실수형으로 변환)
def parse_label(label_str):
    """
    <FLOAT> <|-|><|0|><|.|><|2|><|1|><|3|><|2|> </FLOAT> 형태의 문자열을
    실수형(float) -0.2132 로 변환합니다.
    """
    if not isinstance(label_str, str):
        return None
    
    # <FLOAT> 태그 제거
    clean_str = label_str.replace('<FLOAT>', '').replace('</FLOAT>', '').strip()
    
    # <|x|> 형태에서 값만 추출하여 합침 (정규표현식 사용)
    # 예: <|-|> -> -, <|0|> -> 0
    parsed_chars = re.findall(r'<\|(.*?)\|>', clean_str)
    number_str = "".join(parsed_chars)
    
    try:
        return float(number_str)
    except ValueError:
        return None

# 2. 데이터셋 로드 (경로는 사용자 환경에 맞게 설정)
# 주의: 세 데이터셋의 분자 리스트가 일치하도록 모두 'test' 셋을 사용하는 것을 권장합니다.
base_path = '/home/jovyan/CHJ/Mol-LLM_Custom/dataset/train/'
path_gap = base_path + 'qm9_homo_lumo_gap_subtask-0_test'
path_homo = base_path + 'qm9_homo_subtask-0_test' 
path_lumo = base_path + 'qm9_lumo_subtask-0_test'

print("데이터셋 로드 중...")
ds_gap = load_from_disk(path_gap)
ds_homo = load_from_disk(path_homo)
ds_lumo = load_from_disk(path_lumo)

# 3. 데이터프레임 변환 함수
def dataset_to_df(dataset, value_name):
    # 필요한 컬럼만 추출 (input_mol_string: 식별자, label: 값)
    data = []
    for item in dataset:
        val = parse_label(item['label'])
        mol = item['input_mol_string']
        data.append({'mol_id': mol, value_name: val})
    return pd.DataFrame(data)

# 각 데이터셋을 DataFrame으로 변환
print("DataFrame 변환 및 라벨 파싱 중...")
df_gap = dataset_to_df(ds_gap, 'gap')
df_homo = dataset_to_df(ds_homo, 'homo')
df_lumo = dataset_to_df(ds_lumo, 'lumo')

# 4. 데이터 병합 (분자 문자열 기준 Inner Join)
# 모든 데이터셋에 공통으로 존재하는 분자에 대해서만 검증 수행
df_merged = df_gap.merge(df_homo, on='mol_id').merge(df_lumo, on='mol_id')

print(f"병합된 데이터 개수: {len(df_merged)}개")

# 5. 관계 검증 (Gap = LUMO - HOMO)
# 데이터셋의 Gap 값과 계산된 (LUMO - HOMO) 값의 차이 계산
df_merged['calculated_gap'] = df_merged['lumo'] - df_merged['homo']
df_merged['diff'] = (df_merged['gap'] - df_merged['calculated_gap']).abs()

# 오차 허용 범위 설정 (부동소수점 연산 및 데이터셋 표기 정밀도 고려, 예: 1e-4)
tolerance = 1e-4
match_count = (df_merged['diff'] < tolerance).sum()
match_rate = (match_count / len(df_merged)) * 100

print("-" * 30)
print("검증 결과 Report")
print("-" * 30)
print(f"1. 평균 오차 (Mean Absolute Error): {df_merged['diff'].mean():.6f}")
print(f"2. 허용 오차({tolerance}) 내 일치하는 데이터 수: {match_count} / {len(df_merged)}")
print(f"3. 일치율: {match_rate:.2f}%")

# 불일치 샘플 확인 (상위 5개)
if match_count < len(df_merged):
    print("\n[참고] 불일치 샘플 예시 (Top 5):")
    print(df_merged[df_merged['diff'] >= tolerance][['gap', 'homo', 'lumo', 'calculated_gap', 'diff']].head())
else:
    print("\n모든 데이터가 완벽하게 일치합니다!")

/home/jovyan/miniconda3/envs/MolDA_CHJ/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


데이터셋 로드 중...
DataFrame 변환 및 라벨 파싱 중...
병합된 데이터 개수: 0개
------------------------------
검증 결과 Report
------------------------------
1. 평균 오차 (Mean Absolute Error): nan
2. 허용 오차(0.0001) 내 일치하는 데이터 수: 0 / 0
3. 일치율: nan%

모든 데이터가 완벽하게 일치합니다!


/tmp/ipykernel_633110/631797575.py:69: RuntimeWarning: invalid value encountered in scalar divide
  match_rate = (match_count / len(df_merged)) * 100


In [13]:
train_ds = load_from_disk("/home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_train_3.3M_0415")
val_ds = load_from_disk("/home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_validation_3.3M_0415")
test_ds = load_from_disk("/home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415")


In [14]:
len(train_ds), len(val_ds), len(test_ds)

(2850879, 70932, 70932)

In [15]:
train_ds[0]

{'x': [[5, 0, 4, 5, 3, 0, 2, 0, 0],
  [6, 0, 3, 5, 0, 0, 1, 0, 1],
  [5, 0, 3, 5, 0, 0, 1, 0, 1],
  [7, 0, 1, 5, 0, 0, 1, 0, 0],
  [5, 1, 4, 5, 0, 0, 2, 0, 1],
  [5, 0, 3, 5, 0, 0, 1, 1, 1],
  [5, 0, 3, 5, 1, 0, 1, 1, 1],
  [5, 0, 3, 5, 1, 0, 1, 1, 1],
  [5, 0, 3, 5, 0, 0, 1, 1, 1],
  [7, 0, 2, 5, 0, 0, 1, 0, 0],
  [5, 0, 4, 5, 1, 0, 2, 0, 0],
  [8, 0, 1, 5, 0, 0, 2, 0, 0],
  [8, 0, 1, 5, 0, 0, 2, 0, 0],
  [5, 0, 3, 5, 1, 0, 1, 1, 1],
  [5, 0, 3, 5, 1, 0, 1, 1, 1],
  [5, 0, 3, 5, 0, 0, 1, 1, 1],
  [5, 0, 3, 5, 1, 0, 1, 1, 1],
  [5, 0, 3, 5, 1, 0, 1, 1, 1],
  [5, 0, 3, 5, 0, 0, 1, 1, 1],
  [8, 0, 1, 5, 0, 0, 2, 0, 0],
  [5, 0, 3, 5, 0, 0, 1, 1, 1],
  [5, 0, 2, 5, 0, 0, 0, 0, 0],
  [5, 0, 2, 5, 0, 0, 0, 0, 0],
  [5, 0, 4, 5, 2, 0, 2, 0, 0],
  [5, 0, 4, 5, 2, 0, 2, 0, 0],
  [5, 0, 4, 5, 2, 0, 2, 0, 0],
  [8, 0, 1, 5, 0, 0, 2, 0, 0],
  [5, 0, 3, 5, 1, 0, 1, 1, 1],
  [6, 0, 2, 5, 0, 0, 1, 0, 1],
  [5, 0, 3, 5, 0, 0, 1, 0, 1],
  [6, 0, 3, 5, 2, 0, 1, 0, 0]],
 'edge_index': [[0,
   1,
   1,
 